In [96]:
from pathlib import Path

# pip install sentence-transformers
# pip install tf-keras

In [97]:
def clean_text(text):
    text = text.replace("\n", " ")
    text = " ".join(text.split())
    return text


def chunk_text(text, chunk_size=250, overlap=50):
    words = text.split()
    chunks = []

    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = words[start:end]
        chunks.append(" ".join(chunk))
        start = end - overlap

    return chunks


def load_documents(folder_path):
    documents = []
    for file in Path(folder_path).glob("*.txt"):
        with open(file, "r", encoding="utf-8") as f:
            documents.append(f.read())
    return documents

In [98]:
# Step 1 - Prepare your documents

documents = load_documents("docs")

all_chunks = []

for doc in documents:
    cleaned_doc = clean_text(doc)
    chunks = chunk_text(cleaned_doc)
    all_chunks.extend(chunks)

In [99]:
print(all_chunks)
print(len(all_chunks))

['Google Corp - Q3 2025 Revenue Report - Total Revenue: $120 million (up 15% from Q2 2025) - Revenue by Region: - North America: $60 million - Europe: $35 million - Asia-Pacific: $25 million - Top Products: - Product A: $50 million - Product B: $40 million - Product C: $30 million - Net Profit: $25 million (21% margin) - Key Notes: The increase in revenue was primarily driven by the launch of Product A in North America and strong sales of Product B in Europe.', 'Muhammad Ali Jinnah (born December 25, 1876, in Karachi, British India; died September 11, 1948, in Karachi, Pakistan) was a lawyer, politician, and statesman who became the founder and first Governor-General of Pakistan. He is widely known in Pakistan as Quaid-i-Azam, meaning “Great Leader,” and is regarded as the father of the nation. Jinnah was the eldest of seven children born to Jinnahbhai Poonja, a prosperous merchant, and Mithibai. His family belonged to the Khoja community, which had converted to Islam centuries earlier

In [100]:
# Step 2 - Create embeddings

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [101]:
# Create Embeddings for All Chunks

embeddings = model.encode(all_chunks, convert_to_numpy=True)

In [102]:
embeddings[0]

array([-3.42222564e-02, -9.32644755e-02,  1.77820735e-02, -1.00474492e-01,
       -6.99443445e-02,  4.60172221e-02,  2.47900821e-02,  3.51172388e-02,
        2.10369192e-02,  2.05897372e-02, -2.91072838e-02, -3.35102305e-02,
        2.63781310e-03, -2.10978612e-02,  2.06989013e-02,  1.65054034e-02,
        4.64386307e-02, -6.36276901e-02, -1.98676921e-02, -1.02077685e-01,
        3.87461148e-02, -1.43923191e-02,  7.16883838e-02, -8.75416864e-03,
        5.68573847e-02, -1.62655208e-02, -4.09607589e-02, -5.46139404e-02,
        1.31844860e-02, -3.93425189e-02, -1.09117724e-01,  4.73804399e-02,
        3.23022567e-02,  3.65063399e-02,  2.48070583e-02, -2.25668512e-02,
       -7.63478130e-02,  9.06160276e-04, -1.00922119e-02, -2.53703655e-03,
        1.71148274e-02, -6.03338070e-02, -1.44435734e-01, -5.39860018e-02,
        6.09959252e-02,  1.89252738e-02, -7.80741796e-02,  1.04898617e-01,
        3.47933844e-02,  5.83781861e-02, -3.80888917e-02, -8.25990085e-03,
       -2.34675314e-02, -

In [103]:
print(embeddings.shape)

# (number_of_chunks, 384)

(10, 384)


In [104]:
# Save Embeddings

import numpy as np

np.save("chunk_embeddings.npy", embeddings)

### Step 3 - Build a simple similarity search


In [105]:
# From Step 1 & 2, you already have:

print(all_chunks)  # List[str]
print(embeddings)  # numpy.ndarray, shape = (num_chunks, 384)
print(model)  # SentenceTransformer("all-MiniLM-L6-v2")

['Google Corp - Q3 2025 Revenue Report - Total Revenue: $120 million (up 15% from Q2 2025) - Revenue by Region: - North America: $60 million - Europe: $35 million - Asia-Pacific: $25 million - Top Products: - Product A: $50 million - Product B: $40 million - Product C: $30 million - Net Profit: $25 million (21% margin) - Key Notes: The increase in revenue was primarily driven by the launch of Product A in North America and strong sales of Product B in Europe.', 'Muhammad Ali Jinnah (born December 25, 1876, in Karachi, British India; died September 11, 1948, in Karachi, Pakistan) was a lawyer, politician, and statesman who became the founder and first Governor-General of Pakistan. He is widely known in Pakistan as Quaid-i-Azam, meaning “Great Leader,” and is regarded as the father of the nation. Jinnah was the eldest of seven children born to Jinnahbhai Poonja, a prosperous merchant, and Mithibai. His family belonged to the Khoja community, which had converted to Islam centuries earlier

In [106]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [107]:
# Convert User Question into Embedding
def embed_question(question, model):
    return model.encode([question], convert_to_numpy=True)


# Compute Similarity Scores
def compute_similarity(question_embedding, chunk_embeddings):
    similarities = cosine_similarity(question_embedding, chunk_embeddings)
    return similarities[0]  # flatten array


# Retrieve Top 3 Relevant Chunks
def retrieve_top_chunks(question, chunks, embeddings, model, top_k=3):
    question_embedding = embed_question(question, model)
    similarity_scores = compute_similarity(question_embedding, embeddings)

    top_indices = np.argsort(similarity_scores)[-top_k:][::-1]

    top_chunks = [chunks[i] for i in top_indices]
    return top_chunks

In [108]:
question = "What is name of Quaid-e-azam's father?"

top_chunks = retrieve_top_chunks(
    question=question, chunks=all_chunks, embeddings=embeddings, model=model
)

for i, chunk in enumerate(top_chunks, 1):
    print(f"\n--- Chunk {i} ---\n{chunk}")


--- Chunk 1 ---
Muhammad Ali Jinnah (born December 25, 1876, in Karachi, British India; died September 11, 1948, in Karachi, Pakistan) was a lawyer, politician, and statesman who became the founder and first Governor-General of Pakistan. He is widely known in Pakistan as Quaid-i-Azam, meaning “Great Leader,” and is regarded as the father of the nation. Jinnah was the eldest of seven children born to Jinnahbhai Poonja, a prosperous merchant, and Mithibai. His family belonged to the Khoja community, which had converted to Islam centuries earlier. There has been historical debate regarding Jinnah’s exact date of birth, with some school records suggesting October 20, 1875, although Jinnah himself consistently maintained that he was born on December 25, 1876. Jinnah received his early education in Karachi at the Sind Madrasat al-Islam and later at the Christian Missionary Society High School. At the age of sixteen, he passed the matriculation examination of the University of Bombay. In 189

### Step 4 - Build the RAG prompt


In [109]:
def build_rag_prompt(question, context_chunks):
    context = "\n\n".join(context_chunks)

    prompt = f"""
    You are a helpful assistant. Answer the question **using the context provided**.
    - Be concise and direct.
    - Use only the context, but infer answers if enough information is present.
    - If the answer is not in the context, say "Not enough information in the provided context."

    Context:
    {context}

    Question: {question}
    Answer:
    """

    return prompt


In [110]:
question = "What is name of Quaid-e-azam's father?"

In [111]:
rag_prompt = build_rag_prompt(question, top_chunks)

print(rag_prompt)


    You are a helpful assistant. Answer the question **using the context provided**.
    - Be concise and direct.
    - Use only the context, but infer answers if enough information is present.
    - If the answer is not in the context, say "Not enough information in the provided context."

    Context:
    Muhammad Ali Jinnah (born December 25, 1876, in Karachi, British India; died September 11, 1948, in Karachi, Pakistan) was a lawyer, politician, and statesman who became the founder and first Governor-General of Pakistan. He is widely known in Pakistan as Quaid-i-Azam, meaning “Great Leader,” and is regarded as the father of the nation. Jinnah was the eldest of seven children born to Jinnahbhai Poonja, a prosperous merchant, and Mithibai. His family belonged to the Khoja community, which had converted to Islam centuries earlier. There has been historical debate regarding Jinnah’s exact date of birth, with some school records suggesting October 20, 1875, although Jinnah himself cons

In [112]:
import ollama

response = ollama.chat(
    model="llama3.2", messages=[{"role": "user", "content": rag_prompt}]
)

print(response["message"]["content"])

Jinnahbhai Poonja.


### Step 5 - Test RAG vs No-RAG


In [117]:
questions = [
    "What was the name of Muhammad Ali Jinnah's father?",
    "When and where was Muhammad Ali Jinnah born, and what title is he commonly known by in Pakistan?",
    "What device is used to capture solar energy and convert it into electricity?",
    "What are the three main types of volcanoes, and give one example of each type?",
    "What items were missing or damaged in order #A12345?",
    "On what date was order #A12345 placed?",
    "What resolution does the customer request regarding the damaged item?",
    "What is the customer's name who sent the email?",
    "What was google Corp's total revenue in Q3 2025?",
    "Which product generated the highest revenue, and how much?"
]



In [114]:
# LLaMA Without RAG
def ask_llama_no_rag(question):
    response = ollama.chat(
        model="llama3.2", messages=[{"role": "user", "content": question}]
    )
    return response["message"]["content"]

In [115]:
def ask_llama_with_rag(question, chunks, embeddings, model):
    top_chunks = retrieve_top_chunks(
        question=question, chunks=chunks, embeddings=embeddings, model=model
    )

    prompt = build_rag_prompt(question, top_chunks)

    response = ollama.chat(
        model="llama3.2", messages=[{"role": "user", "content": prompt}]
    )
    return response["message"]["content"]


# sample usage

# answer = ask_llama_with_rag(
#     question=questions[0], chunks=all_chunks, embeddings=embeddings, model=model
# )
# answer

In [118]:
with open("rag_qa_results.txt", "w", encoding="utf-8") as f:
    for idx, question in enumerate(questions, start=1):
        try:
            answer = ask_llama_with_rag(
                question=question, chunks=all_chunks, embeddings=embeddings, model=model
            )

            # Write question and answer to file
            f.write(f"Question {idx}: {question}\n")
            f.write(f"Answer {idx}: {answer}\n")
            f.write("-" * 80 + "\n")

            # Print Q&A to console
            print(f"Question {idx}: {question}")
            print(f"Answer {idx}: {answer}")
            print("-" * 80)

        except Exception as e:
            f.write(f"Question {idx}: {question}\n")
            f.write(f"Answer {idx}: Error occurred - {e}\n")
            f.write("-" * 80 + "\n")

            # Print error to console
            print(f"Question {idx}: {question}")
            print(f"Answer {idx}: Error occurred - {e}")
            print("-" * 80)

print("RAG Q&A results saved to 'rag_qa_results.txt'.")


Question 1: What was the name of Muhammad Ali Jinnah's father?
Answer 1: Muhammad Ali Jinnah's father was Jinnahbhai Poonja.
--------------------------------------------------------------------------------
Question 2: When and where was Muhammad Ali Jinnah born, and what title is he commonly known by in Pakistan?
Answer 2: Muhammad Ali Jinnah was born on December 25, 1876, in Karachi, British India. He is widely known in Pakistan as Quaid-i-Azam, meaning “Great Leader.”
--------------------------------------------------------------------------------
Question 3: What device is used to capture solar energy and convert it into electricity?
Answer 3: The photovoltaic (PV) cells in solar panels are used to capture solar energy and convert it into electricity through the photovoltaic effect.
--------------------------------------------------------------------------------
Question 4: What are the three main types of volcanoes, and give one example of each type?
Answer 4: The three main types 

In [119]:
# File to store answers from LLaMA without RAG
with open("llama_no_rag_results.txt", "w", encoding="utf-8") as f:
    for idx, question in enumerate(questions, start=1):
        try:
            # Get answer from LLaMA without RAG
            answer = ask_llama_no_rag(question)

            # Write question and answer to file
            f.write(f"Question {idx}: {question}\n")
            f.write(f"Answer {idx}: {answer}\n")
            f.write("-" * 80 + "\n")

            # Also print to console
            print(f"Question {idx}: {question}")
            print(f"Answer {idx}: {answer}")
            print("-" * 80)

        except Exception as e:
            # Log errors and continue
            f.write(f"Question {idx}: {question}\n")
            f.write(f"Answer {idx}: Error occurred - {e}\n")
            f.write("-" * 80 + "\n")

            print(f"Question {idx}: {question}")
            print(f"Answer {idx}: Error occurred - {e}")
            print("-" * 80)

print(f"LLaMA no-RAG Q&A results saved to `llama_no_rag_results.txt`.")


Question 1: What was the name of Muhammad Ali Jinnah's father?
Answer 1: The name of Muhammad Ali Jinnah's father is Mian Ghulam Muhammad, however, some sources also mention his paternal grandfather as Allama Muhammad Iqbal.
--------------------------------------------------------------------------------
Question 2: When and where was Muhammad Ali Jinnah born, and what title is he commonly known by in Pakistan?
Answer 2: Muhammad Ali Jinnah was born on December 25, 1876, in Mumbai (then known as Bombay), British India. He is commonly known as "Quaid-e-Azam" or "Great Leader", which is the highest honorific title in Pakistan.
--------------------------------------------------------------------------------
Question 3: What device is used to capture solar energy and convert it into electricity?
Answer 3: The device used to capture solar energy and convert it into electricity is called a Solar Panel. Solar panels are made up of photovoltaic (PV) cells, which are made from semiconducting ma